In [13]:
SHOULD_PRINT = False

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm

In [15]:
import torch
torch.manual_seed(32)

In [16]:
from torch.utils.data import Dataset 

class CustomDataset(Dataset):
    def __init__(self, filename):
        self.data = []
        with open(filename, 'r') as f:
            for line in f:
                numbers = [float(num) for num in line.strip().split()]
                self.data.append((numbers[:4], numbers[4:]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs, targets = self.data[idx]
        return torch.tensor(inputs), torch.tensor(targets)

In [17]:
filename = "/kaggle/input/harmonic/sph_100_10_20.txt"
full_dataset = CustomDataset(filename)

train_size = int(0.8 * len(full_dataset))
rest_size = len(full_dataset) - train_size
val_size = rest_size // 2
test_size = rest_size // 2
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(32))

In [18]:
from torch.utils.data import DataLoader

BATCH_SIZE = 1024
train_shuffle = True
val_shuffle = False

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=train_shuffle)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=val_shuffle)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [19]:
for inputs, targets in train_dataloader:
    assert inputs.shape[1] == 4 and targets.shape[1] == 2

In [46]:
import torch.nn as nn
import torch.nn.init as init

# Define your autoencoder model
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU()
        )
        self.bottleneck = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size),
        )
        self.__init_weights()

    def forward(self, x):
        x = self.encoder(x)
        x = self.bottleneck(x)
        return x
    
    def __init_weights(self):
        # Initialize linear layers using Kaiming (He) uniform initialization
        for layer in [self.encoder, self.bottleneck]:
            for m in layer.children():
                if isinstance(m, nn.Linear):
                    init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='tanh')
                    if m.bias is not None:
                        init.zeros_(m.bias)
    
class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )
        self.__init_weights()

    def forward(self, x):
        x = self.decoder(x)
        return x
    
    def __init_weights(self):
        # Initialize linear layers using Kaiming (He) uniform initialization
        for layer in [self.decoder]:
            for m in layer.children():
                if isinstance(m, nn.Linear):
                    init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='tanh')
                    if m.bias is not None:
                        init.zeros_(m.bias)
    
    
class RegressionAutoencoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(RegressionAutoencoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

### Train encoder

In [28]:
from torch.utils.tensorboard import SummaryWriter

def encoder_train(model, dataloader, optimizer, loss_fn, epoch, writer, log_perc = 0.1):
    model.train()
    total_loss = 0
    total_diff = 0
    best_diff = 100

    logs_steps = int(log_perc * len(dataloader))
    start_step = epoch * len(dataloader)

    before_lr = optimizer.param_groups[0]["lr"]
    writer.add_scalar('Lr/Encoder Train', before_lr, epoch)
    for idx, (inputs, _) in enumerate(dataloader):
        inputs = inputs.to(device)
        
        optimizer.zero_grad()

        outputs = model(inputs)

        loss = loss_fn(outputs, inputs)
        diff = torch.abs(outputs - inputs).mean()

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_diff += diff.item()
        
        if idx % logs_steps == 0:
            writer.add_scalar('Loss/Encoder Train', loss.item(), start_step + idx)
            writer.add_scalar('Absolute Difference/Encoder Train', diff.item(), start_step + idx)
            
            if SHOULD_PRINT:
                print(f"Loss/Encoder Train: {loss.item()}")
                print(f"Absolute Difference/Encoder Train: {diff.item()}")

    #scheduler.step()
    after_lr = optimizer.param_groups[0]["lr"]
    average_loss = total_loss / len(dataloader)
    average_diff = total_diff / len(dataloader)
    
    writer.add_scalar('Avg Loss/Encoder Train', average_loss, epoch)
    writer.add_scalar('Avg Absolute Difference/Encoder Train', average_diff, epoch)
    writer.add_scalar('Lr/Encoder Train', after_lr, epoch)
    
    if SHOULD_PRINT:
        print(f"Avg Loss/Encoder Train: {average_loss}")
        print(f"Absolute Difference/Encoder Train: {average_diff}")
        print(f"Lr/Encoder Train: {after_lr}")

In [35]:
def encoder_validate(model, dataloader, loss_fn, epoch, writer):
    model.eval()
    total_loss = 0
    total_diff = 0

    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)

            loss = loss_fn(outputs, inputs)
            diff = torch.abs(outputs - inputs).mean()
            
            total_loss += loss.item()
            total_diff += diff.item()

    average_loss = total_loss / len(dataloader)
    average_diff = total_diff / len(dataloader)

    if writer is not None:
        writer.add_scalar('Avg Loss/Encoder Val', average_loss, epoch)
        writer.add_scalar('Avg Absolute Difference/Encoder Val', average_diff, epoch)
    
    if SHOULD_PRINT:
        print(f"Avg Loss/Encoder Val: {average_loss}")
        print(f"Avg Absolute Difference/Encoder Val: {average_diff}")

    if epoch is not None:
        print(f"Epoch [{epoch+1}/{Encoder_NUM_EPOCHS}] Encoder Val Loss: {average_loss:.4f}, Val Diff: {average_diff:.15f}")
    else:
        print(f"Test Loss: {average_loss:.4f}, Encoder Test Diff: {average_diff:.15f}")
        
    return average_diff

In [36]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"

In [37]:
Encoder_LR = 5e-4 # too big
Encoder_NUM_EPOCHS = 1000
Encoder_MOMENTUM = 0.9
Encoder_WEIGHT_DECAY = 0.99
Encoder_MOMENTUM = 0.9
Encoder_VALIDATION_STEPS = int(Encoder_NUM_EPOCHS * 0.1)

In [38]:
Encoder_VALIDATION_STEPS

100

In [39]:
encoder = Encoder(4, 256)

encoder_loss_fn = nn.MSELoss()
encoder_optimizer = optim.Adam(encoder.parameters(), lr=Encoder_LR)

In [40]:
import datetime

now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(f"tb_logs/encoder/{now}")

encoder = encoder.to(device)
best_avg_diff = 1000
for idx, epoch in enumerate(tqdm(range(Encoder_NUM_EPOCHS))):
    encoder_train(encoder, train_dataloader, encoder_optimizer, encoder_loss_fn, epoch, writer)

    if idx % Encoder_VALIDATION_STEPS == 0:
        average_diff = encoder_validate(encoder, val_dataloader, encoder_loss_fn, epoch, writer)
        if average_diff < best_avg_diff:
            best_avg_diff = average_diff
            torch.save({
                'epoch': epoch,
                'model_state_dict': encoder.state_dict(),
                'optimizer_state_dict': encoder_optimizer.state_dict(),
                'loss': encoder_loss_fn,
                }, f"encoder_checkpoint-nosch-diff-{average_diff}.pth")
            
average_diff = encoder_validate(encoder, val_dataloader, encoder_loss_fn, epoch, writer)
if average_diff < best_avg_diff:
    best_avg_diff = average_diff
    torch.save({
        'epoch': epoch,
        'model_state_dict': encoder.state_dict(),
        'optimizer_state_dict': encoder_optimizer.state_dict(),
        'loss': encoder_loss_fn,
        }, f"encoder_checkpoint-nosch-diff-{average_diff}.pth")

  0%|          | 1/1000 [00:00<05:35,  2.98it/s]

Epoch [1/1000] Encoder Val Loss: 6.5650, Val Diff: 1.961948633193970


 10%|█         | 101/1000 [00:33<05:24,  2.77it/s]

Epoch [101/1000] Encoder Val Loss: 0.0004, Val Diff: 0.014144174133738


 20%|██        | 201/1000 [01:06<04:25,  3.01it/s]

Epoch [201/1000] Encoder Val Loss: 0.0024, Val Diff: 0.043026874462763


 30%|███       | 301/1000 [01:40<03:42,  3.14it/s]

Epoch [301/1000] Encoder Val Loss: 0.0004, Val Diff: 0.015002454010149


 33%|███▎      | 330/1000 [01:49<03:43,  3.00it/s]


KeyboardInterrupt: 

### Train model

In [47]:
from torch.utils.tensorboard import SummaryWriter

def train(model, dataloader, optimizer, loss_fn, epoch, writer, log_perc = 0.1):
    model.train()
    total_loss = 0
    total_diff = 0
    best_diff = 100

    logs_steps = int(log_perc * len(dataloader))
    start_step = epoch * len(dataloader)

    before_lr = optimizer.param_groups[0]["lr"]
    writer.add_scalar('Lr/Train', before_lr, epoch)
    for idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()

        outputs = model(inputs)

        loss = loss_fn(outputs, targets)
        diff = torch.abs(outputs - targets).mean()

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_diff += diff.item()
        
        if idx % logs_steps == 0:
            writer.add_scalar('Loss/Train', loss.item(), start_step + idx)
            writer.add_scalar('Absolute Difference/Train', diff.item(), start_step + idx)
            
            if SHOULD_PRINT:
                print(f"Loss/Train: {loss.item()}")
                print(f"Absolute Difference/Train: {diff.item()}")

    #scheduler.step()
    after_lr = optimizer.param_groups[0]["lr"]
    average_loss = total_loss / len(dataloader)
    average_diff = total_diff / len(dataloader)
    
    writer.add_scalar('Avg Loss/Train', average_loss, epoch)
    writer.add_scalar('Avg Absolute Difference/Train', average_diff, epoch)
    writer.add_scalar('Lr/Train', after_lr, epoch)
    
    if SHOULD_PRINT:
        print(f"Avg Loss/Train: {average_loss}")
        print(f"Absolute Difference/Train: {average_diff}")
        print(f"Lr/Train: {after_lr}")

In [48]:
def validate(model, dataloader, loss_fn, epoch, writer):
    model.eval()
    total_loss = 0
    total_diff = 0

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)

            loss = loss_fn(outputs, targets)
            diff = torch.abs(outputs - targets).mean()
            
            total_loss += loss.item()
            total_diff += diff.item()

    average_loss = total_loss / len(dataloader)
    average_diff = total_diff / len(dataloader)

    if writer is not None:
        writer.add_scalar('Avg Loss/Val', average_loss, epoch)
        writer.add_scalar('Avg Absolute Difference/Val', average_diff, epoch)
    
    if SHOULD_PRINT:
        print(f"Avg Loss/Val: {average_loss}")
        print(f"Avg Absolute Difference/Val: {average_diff}")

    if epoch is not None:
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Val Loss: {average_loss:.4f}, Val Diff: {average_diff:.15f}")
    else:
        print(f"Test Loss: {average_loss:.4f}, Test Diff: {average_diff:.15f}")
        
    return average_diff

In [49]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"

In [53]:
LR = 5e-5
NUM_EPOCHS = 1000
MOMENTUM = 0.9
WEIGHT_DECAY = 0.99
MOMENTUM = 0.9
VALIDATION_STEPS = int(NUM_EPOCHS * 0.1)

In [54]:
VALIDATION_STEPS

100

In [55]:
decoder = Decoder(4, 256, 2)
model = RegressionAutoencoder(encoder, decoder)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [56]:
import datetime

now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(f"tb_logs/model/{now}")

model = model.to(device)
best_avg_diff = 1000
for idx, epoch in enumerate(tqdm(range(NUM_EPOCHS))):
    train(model, train_dataloader, optimizer, loss_fn, epoch, writer)

    if idx % VALIDATION_STEPS == 0:
        average_diff = validate(model, val_dataloader, loss_fn, epoch, writer)
        if average_diff < best_avg_diff:
            best_avg_diff = average_diff
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss_fn,
                }, f"checkpoint-nosch-diff-{average_diff}.pth")
            
average_diff = validate(model, val_dataloader, loss_fn, epoch, writer)
if average_diff < best_avg_diff:
    best_avg_diff = average_diff
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss_fn,
        }, f"checkpoint-nosch-diff-{average_diff}.pth")

  0%|          | 1/1000 [00:00<06:09,  2.70it/s]

Epoch [1/1000] Val Loss: 0.0998, Val Diff: 0.243626510103544


 10%|█         | 101/1000 [00:36<05:20,  2.81it/s]

Epoch [101/1000] Val Loss: 0.0406, Val Diff: 0.132855425278346


 20%|██        | 201/1000 [01:12<04:41,  2.84it/s]

...

Epoch [801/1000] Val Loss: 0.0395, Val Diff: 0.128067533175151


 90%|█████████ | 901/1000 [05:26<00:34,  2.86it/s]

Epoch [901/1000] Val Loss: 0.0399, Val Diff: 0.130320549011230


100%|██████████| 1000/1000 [06:01<00:00,  2.76it/s]

Epoch [1000/1000] Val Loss: 0.0392, Val Diff: 0.126322897771994
